In [1]:
import numpy as np
import random
import syft as sy
import json
from syft.core.frameworks.torch import utils

In [2]:
x = sy.array([1,2,3,4])
me = sy.local_worker
hook = me.hook

In [3]:
bob = sy.VirtualWorker(hook=hook,id=2)

In [4]:
x = sy.array([1,2,3,4])

In [5]:
xt = x.send(bob)
# xt = x.torch().send(bob)

In [9]:
cmd, locations, owners = utils.compile_command(attr="torch", args=(xt), kwargs={}, has_self=True)
cmd

{'mode': 'subscribe',
 'obj': {'args': {'data': ['data is remote'],
   'id': 98581135765,
   'id_at_location': 156724394,
   'location': 2,
   'type': 'numpy.array_ptr'},
  'command': 'torch',
  'has_self': True,
  'kwargs': {},
  'self': None}}

In [12]:
me.send_torch_command(recipient=xt.location, message=cmd)

{'command': 'torch', 'has_self': True, 'args': {'type': 'numpy.array_ptr', 'id': 98581135765, 'data': ['data is remote'], 'location': 2, 'id_at_location': 156724394}, 'kwargs': {}, 'self': None}


RuntimeError: Command "torch" is not a supported Torch operation.

In [8]:
xptr = x.send(bob)

message wrapper json
b'{"message": {"obj": {"type": "numpy.array", "id": 9550542501, "data": [1, 2, 3, 4]}, "mode": "acquire"}, "type": "obj"}\n'
{'message': array([1, 2, 3, 4]), 'type': 'obj'}


In [23]:
xptr

array_ptr(['data is remote'], dtype='<U14')

In [20]:
x = xptr.get()

In [3]:
x

array([1, 2, 3, 4])